In [ ]:
import os
import sys
from typing import List, Dict, Any

import uvicorn
import vertexai
from dotenv import load_dotenv
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from vertexai import agent_engines
from vertexai.preview import reasoning_engines
import uuid

# It's assumed you have your adk_short_bot agent in a local directory
# This is a placeholder import. Replace with your actual agent import.
# try:
#     from adk_short_bot.agent import root_agent
# except ImportError:
#     print("Warning: Could not import 'root_agent'. Using a placeholder.")
#     print("Please ensure 'adk_short_bot' is in your Python path.")
#     # Define a placeholder if the real one isn't available to make the script runnable
#     class PlaceholderAgent:
#         def __call__(self, *args, **kwargs):
#             return "This is a placeholder agent response."
#     root_agent = PlaceholderAgent()


# --- FastAPI App Initialization ---
# Initialize the FastAPI app with metadata for the documentation
app = FastAPI(
    title="Vertex AI Agent Interaction API",
    description="An API to list deployments, create sessions, and interact with existing Vertex AI Agents.",
    version="1.0.0",
)

# --- Vertex AI Initialization ---
# Load environment variables from a .env file for local development
load_dotenv()

# Get project configuration from environment variables
PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")
LOCATION = os.getenv("GOOGLE_CLOUD_LOCATION")
BUCKET = os.getenv("GOOGLE_CLOUD_STAGING_BUCKET")

# A single startup event to initialize Vertex AI and check for required variables
@app.on_event("startup")
def startup_event():
    """Initializes Vertex AI SDK on application startup."""
    if not all([PROJECT_ID, LOCATION, BUCKET]):
        print(
            "FATAL ERROR: Missing required environment variables: "
            "GOOGLE_CLOUD_PROJECT, GOOGLE_CLOUD_LOCATION, GOOGLE_CLOUD_STAGING_BUCKET"
        )
        # In a real app, you might want a more graceful shutdown
        sys.exit(1)

    print("Initializing Vertex AI...")
    vertexai.init(
        project=PROJECT_ID,
        location=LOCATION,
        staging_bucket=BUCKET,
    )
    print("Vertex AI Initialized Successfully.")

# --- Pydantic Models for Request & Response ---
# These models define the expected structure of your API's JSON data.
# FastAPI uses them for validation, serialization, and documentation.

class DeploymentResponse(BaseModel):
    resource_name: str = Field(..., example="projects/your-project/locations/us-central1/agentEngines/12345")

class SessionRequest(BaseModel):
    user_id: str = Field(..., example="test-user-123")
    resource_id: str = Field(..., example="123456689")

class SessionResponse(BaseModel):
    session_id: str = Field(..., example="4768534100908703744")
    user_id: str = Field(..., example="a123")
    resource_id: str = Field(..., example="3976792820177436672")
    last_update_time: str = Field(..., example="1750166696.884852")

class MessageRequest(BaseModel):
    user_id: str = Field(..., example="abc1")
    resource_id: str = Field(..., example="3976792820177436672")
    session_id: str = Field(..., example="3217044029279567872")
    msg_req: str = Field(..., example="Hi how are you")

class MessageResponse(BaseModel):
    response: List[Dict[str, Any]] = Field(..., example=[{"chunk": {"text": "hru today?"}}])


# --- Agent Core Functions ---
# These functions contain the core logic for interacting with the Vertex AI Agent Engine.
# They are called by the API endpoints.

def list_all_deployments() -> List[Dict[str, str]]:
    """Lists all deployments."""
    deployments = agent_engines.list()
    return [{"resource_name": dep.resource_name} for dep in deployments]


def create_new_session(resource_id: str, user_id: str) -> Dict[str, Any]:
    """Creates a new session for a user."""
    remote_app = agent_engines.get(resource_id)
    return remote_app.create_session(user_id=user_id)


def send_agent_message(
    resource_id: str, user_id: str, session_id: str, message: str
) -> List[Dict[str, Any]]:
    """Sends a message to the agent and streams the response."""
    try:
        print(f"Sending message with params: resource_id={resource_id}, user_id={user_id}, session_id={session_id}, message={message}")
        remote_app = agent_engines.get(resource_id)
        print("Got remote app:", remote_app)
        
        # Generate a unique invocation ID
        invocation_id = str(uuid.uuid4())
        print("Generated invocation_id:", invocation_id)
        
        events = []
        for event in remote_app.stream_query(
            user_id=user_id,
            session_id=session_id,
            message=message,
            invocation_id=invocation_id
        ):
            print("Event:", event)
            events.append(event)

        print("Response events:", events)
        return events
    except Exception as e:
        print("Error in send_agent_message:", str(e))
        raise e


# --- FastAPI Endpoints ---

@app.get("/deployments", response_model=List[DeploymentResponse], tags=["Deployments"])
def handle_list_deployments():
    """
    Lists all available agent deployments.
    """
    try:
        return list_all_deployments()
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/deployments/create/sessions", response_model=SessionResponse, tags=["Sessions"])
def handle_create_session(session_req: SessionRequest):
    """
    Creates a new chat session for a given deployment.
    """
    try:
        print("Creating session with:", session_req.dict())
        session = create_new_session(session_req.resource_id, session_req.user_id)
        print("Raw session response:", session)
        
        # Map the response fields to match our SessionResponse model
        return {
            "session_id": str(session["id"]),
            "user_id": str(session["userId"]),
            "resource_id": str(session["appName"]),
            "last_update_time": str(session["lastUpdateTime"])
        }
    except Exception as e:
        print("Error creating session:", str(e))
        print("Error type:", type(e))
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/deployments/sessions/send", response_model=MessageResponse, tags=["Sessions"])
def handle_send_message(msg_req: MessageRequest):
    """
    Sends a message to a specific session and gets the agent's response.
    """
    try:
        print("Received message request:", msg_req.model_dump())  # Updated to use model_dump instead of dict
        response_events = send_agent_message(
            msg_req.resource_id,
            msg_req.user_id,
            msg_req.session_id,
            msg_req.msg_req
        )
        print("Final response events:", response_events)
        return {"response": response_events}
    except Exception as e:
        print("Error in handle_send_message:", str(e))
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    # To run this FastAPI app:
    # 1. Make sure you have FastAPI and Uvicorn installed:
    #    pip install fastapi "uvicorn[standard]"
    # 2. Set the required environment variables in your terminal or a .env file.
    # 3. Run the script from your terminal:
    #    uvicorn your_filename:app --reload
    #
    # The API will be available at http://127.0.0.1:8000
    # The interactive documentation will be at http://127.0.0.1:8000/docs
    uvicorn.run(app, host="127.0.0.1", port=8000)


In [1]:
import os
import sys
from typing import List, Dict, Any

import uvicorn
import vertexai
from dotenv import load_dotenv
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from vertexai import agent_engines
from vertexai.preview import reasoning_engines
import uuid

In [2]:
load_dotenv()

# Get project configuration from environment variables
PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")
LOCATION = os.getenv("GOOGLE_CLOUD_LOCATION")
BUCKET = os.getenv("GOOGLE_CLOUD_STAGING_BUCKET")
print(PROJECT_ID, LOCATION, BUCKET)

"""Initializes Vertex AI SDK on application startup."""
if not all([PROJECT_ID, LOCATION, BUCKET]):
    print(
        "FATAL ERROR: Missing required environment variables: "
        "GOOGLE_CLOUD_PROJECT, GOOGLE_CLOUD_LOCATION, GOOGLE_CLOUD_STAGING_BUCKET"
    )
    # In a real app, you might want a more graceful shutdown
    sys.exit(1)

print("Initializing Vertex AI...")
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=BUCKET,
)
print("Vertex AI Initialized Successfully.")

g2m-dev us-central1 gs://ask-g2m-agent
Initializing Vertex AI...
Vertex AI Initialized Successfully.


In [3]:
def list_all_deployments() -> List[Dict[str, str]]:
    """Lists all deployments."""
    deployments = agent_engines.list()
    return [{"resource_name": dep.resource_name} for dep in deployments]


def create_new_session(resource_id: str, user_id: str) -> Dict[str, Any]:
    """Creates a new session for a user."""
    remote_app = agent_engines.get(resource_id)
    return remote_app.create_session(user_id=user_id)


def send_agent_message(
    resource_id: str, user_id: str, session_id: str, message: str
) -> List[Dict[str, Any]]:
    """Sends a message to the agent and streams the response."""
    try:
        print(f"Sending message with params: resource_id={resource_id}, user_id={user_id}, session_id={session_id}, message={message}")
        remote_app = agent_engines.get(resource_id)
        print("Got remote app:", remote_app)
        
        # Generate a unique invocation ID
        invocation_id = str(uuid.uuid4())
        print("Generated invocation_id:", invocation_id)
        
        events = []
        for event in remote_app.stream_query(
            user_id=user_id,
            session_id=session_id,
            message=message,
            invocation_id=invocation_id
        ):
            print("Event:", event)
            events.append(event)

        print("Response events:", events)
        return events
    except Exception as e:
        print("Error in send_agent_message:", str(e))
        raise e


In [4]:
list_all_deployments()

[{'resource_name': 'projects/469937863197/locations/us-central1/reasoningEngines/603878174253645824'},
 {'resource_name': 'projects/469937863197/locations/us-central1/reasoningEngines/3976792820177436672'}]

In [5]:
create_new_session("603878174253645824", "123")

{'appName': '603878174253645824',
 'events': [],
 'state': {},
 'id': '6735621975651123200',
 'userId': '123',
 'lastUpdateTime': 1750232771.310405}

In [6]:
send_agent_message("603878174253645824","123","6735621975651123200","Hello what can you do for me?")

Sending message with params: resource_id=603878174253645824, user_id=123, session_id=6735621975651123200, message=Hello what can you do for me?
Got remote app: <vertexai.agent_engines._agent_engines.AgentEngine object at 0x000001362676BBB0> 
resource name: projects/g2m-dev/locations/us-central1/reasoningEngines/603878174253645824
Generated invocation_id: f1416b49-5929-4b3c-b030-95cc2b922473
Response events: []


[]

In [1]:
from vertexai import agent_engines

adk_app = agent_engines.get("7737579984008511488")

In [3]:
session = adk_app.create_session(user_id="USER")

In [4]:
adk_app.list_sessions(user_id="USER")

{'sessions': [{'appName': '7737579984008511488',
   'events': [],
   'state': {},
   'id': '4768674838397059072',
   'lastUpdateTime': 1750253356.219827,
   'userId': 'USER'}]}

In [5]:
session = adk_app.get_session(user_id="USER", session_id="4768674838397059072")

In [6]:
session

{'appName': '7737579984008511488',
 'events': [],
 'state': {},
 'id': '4768674838397059072',
 'lastUpdateTime': 1750253356.219827,
 'userId': 'USER'}

In [10]:
for event in adk_app.stream_query(
    user_id="USER_ID1",
    session_id="5401430586042613760",  # Optional
    message="Using the TxDMV Self-Evaluation Report, can you create a grouped table of all TxDMV programs that includes their FTEs, FY16 budget, delivery model, primary stakeholders, statutory authority, technology systems used, funding source, public safety impact, and regulatory risk??",
):
  print(event)

{'invocation_id': 'e-82a524de-f291-4ae1-835d-3d9d1e5e68a0', 'author': 'SimpleRouter', 'actions': {'state_delta': {'r_user_query': 'Using the TxDMV Self-Evaluation Report, can you create a grouped table of all TxDMV programs that includes their FTEs, FY16 budget, delivery model, primary stakeholders, statutory authority, technology systems used, funding source, public safety impact, and regulatory risk??'}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'decPzt7U', 'timestamp': 1750253602.722282}
{'invocation_id': 'e-82a524de-f291-4ae1-835d-3d9d1e5e68a0', 'author': 'ElasticSearcher', 'actions': {'state_delta': {'elastic_data': "Source: ❓ User Question Examples & AI-Generated Answer Links for Ask g2m.docx (Page: Unknown, Score: 42.09)\nContent: Q7: Using the TxDMV Self-Evaluation Report, can you create a grouped table of all TxDMV programs that includes their FTEs, FY16 budget, delivery model, primary stakeholders, statutory authority, technology systems used, funding source,

In [48]:
event["content"]["parts"][0]["text"]

'Information Not Found.\n## Sources Used\nInformation Not Found'

In [39]:
import json

def extract_final_response(events_data: list) -> str:
    """
    Extracts the final text response from a list of ADK agent events.

    The function looks for the event generated by the final "model" role,
    which is typically the AnswerSynthesizer, and extracts its text content.

    Args:
        events_data (list): A list of dictionaries, where each dictionary is an event.

    Returns:
        str: The extracted text response, or an error message if not found.
    """
    # Iterate backwards as the final response is usually one of the last events
    for event in reversed(events_data):
        # The final response is in an event with a 'content' field 
        # and a 'role' of 'model'.
        if 'content' in event and event.get('content', {}).get('role') == 'model':
            try:
                # Navigate the nested structure to get the text
                return event['content']['parts'][0]['text']
            except (KeyError, IndexError, TypeError) as e:
                return f"Found the response object, but failed to parse it: {e}"
                
    return "Final response not found in the provided events."

# --- Example using the data you provided ---

# Your provided JSON data as a Python list of dictionaries
provided_data = [
    {
        "invocation_id": "e-7fd4666e-d656-4752-8c31-4e5e8cf122bb",
        "author": "SimpleRouter",
        "actions": {
            "state_delta": {
                "r_user_query": "Hi how are you"
            },
            "artifact_delta": {},
            "requested_auth_configs": {}
        },
        "id": "JinwETlU",
        "timestamp": 1750242860.375373
    },
    {
        "invocation_id": "e-7fd4666e-d656-4752-8c31-4e5e8cf122bb",
        "author": "ElasticSearcher",
        "actions": {
            "state_delta": {
                "elastic_data": "An error occurred during Elasticsearch search: AuthenticationException(401, 'security_exception', 'missing authentication credentials for REST request [/ask-g2m-serverless/_search]')"
            },
            "artifact_delta": {},
            "requested_auth_configs": {}
        },
        "id": "Om8NABRb",
        "timestamp": 1750242860.814411
    },
    {
        "invocation_id": "e-7fd4666e-d656-4752-8c31-4e5e8cf122bb",
        "author": "PineconeSearcher",
        "actions": {
            "state_delta": {
                "pinecone_data": "An error occurred during Pinecone search: You haven't specified an API key. Please either set the PINECONE_API_KEY environment variable or pass the 'api_key' keyword argument to the Pinecone client constructor."
            },
            "artifact_delta": {},
            "requested_auth_configs": {}
        },
        "id": "XiaDkPOP",
        "timestamp": 1750242860.547103
    },
    {
        "content": {
            "parts": [
                {
                    "text": "I am an AI and do not have personal feelings. How may I assist you with information from the provided documents?"
                }
            ],
            "role": "model"
        },
        "usage_metadata": {
            "candidates_token_count": 23,
            "prompt_token_count": 390,
            "total_token_count": 464
        },
        "invocation_id": "e-7fd4666e-d656-4752-8c31-4e5e8cf122bb",
        "author": "AnswerSynthesizer",
        "actions": {
            "state_delta": {},
            "artifact_delta": {},
            "requested_auth_configs": {}
        },
        "id": "zv3VLNgI",
        "timestamp": 1750242861.112421
    }
]

# Call the function with your data
final_text = extract_final_response(provided_data)

# Print the result
print(final_text)

I am an AI and do not have personal feelings. How may I assist you with information from the provided documents?


In [50]:
from google.cloud import secretmanager

In [51]:
client = secretmanager.SecretManagerServiceClient()

In [58]:
client.access_secret_version(request={"name":"projects/g2m-dev/secrets/PINECONE_API_KEY/versions/latest"}).payload.data.decode("UTF-8").strip()

'pcsk_4R1jJz_7VwnaJSom8S1H2mh8bbJoWyVbD2f8Z6nUYHjZbT2Xq2GApPYrGjEnmct3WxkHU7'

In [57]:
client.access_secret_version(request={"name":"projects/469937863197/secrets/PINECONE_API_KEY/versions/latest"}).payload.data.decode("UTF-8").strip()

'pcsk_4R1jJz_7VwnaJSom8S1H2mh8bbJoWyVbD2f8Z6nUYHjZbT2Xq2GApPYrGjEnmct3WxkHU7'